In [ ]:
def dic2arg(config:dict):
  arg = ''
  for value in config:
    arg += f'{value if str(config[value]) != "False" else ""} {"" if type(config[value]) == bool else config[value]} '
  return arg

#wan
model_train_list = {
  "wan_t2v_13": "https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/blob/main/split_files/diffusion_models/wan2.1_t2v_1.3B_fp16.safetensors",
  "wan_t2v": "https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/blob/main/split_files/diffusion_models/wan2.1_t2v_14B_fp8_e4m3fn.safetensors",
  "wan_i2v": "https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/blob/main/split_files/diffusion_models/wan2.1_i2v_720p_14B_fp8_e4m3fn.safetensors",
  "vae_wan": "https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/blob/main/split_files/vae/wan_2.1_vae.safetensors",
  "tt5_clip": "https://huggingface.co/Wan-AI/Wan2.1-T2V-1.3B/blob/main/models_t5_umt5-xxl-enc-bf16.pth",
  "wan_clip_vision": "https://huggingface.co/alibaba-pai/Wan2.1-Fun-14B-Control/blob/main/models_clip_open-clip-xlm-roberta-large-vit-huge-14.pth",
  "hun_t2v": "https://huggingface.co/tencent/HunyuanVideo/blob/main/hunyuan-video-t2v-720p/transformers/mp_rank_00_model_states_fp8.pt",
  "hun_i2v": "https://huggingface.co/tencent/HunyuanVideo-I2V/blob/main/hunyuan-video-i2v-720p/transformers/mp_rank_00_model_states.pt",
  "hun_clip_vision": "https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/blob/main/split_files/clip_vision/llava_llama3_vision.safetensors",
  "hun_text_encoder1": "https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/blob/main/split_files/text_encoders/llava_llama3_fp16.safetensors",
  "hun_text_encoder2": "https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/blob/main/split_files/text_encoders/clip_l.safetensors",
  "vae_hun": "https://huggingface.co/tencent/HunyuanVideo/blob/main/hunyuan-video-t2v-720p/vae/pytorch_model.pt"
}

model_folder = os.path.join(root_dir,"models")



In [ ]:
def dic2arg(config:dict):
  arg = ''
  for value in config:
    arg += f'{value if str(config[value]) != "False" else ""} {"" if type(config[value]) == bool else config[value]} '
  return arg

def hug_down(link,path):
  name = path.split('/')[-1]
  folder = path.split(name)[0]
  if "blob" in link:
    link = link.replace("blob","resolve")
  !aria2c --console-log-level=error --summary-interval=10 -c -x 16 -s 16 -k 1M {link} -d {folder} -o {name}

def model_path(model):
  model_name = model
  model_name = model_train_list[model].split('/')[-1]
  model_path = os.path.join(model_folder,model_name)
  if not os.path.exists(model_path):
    hug_down(model_train_list[model],model_path)
  return model_path

op = {
    '--mixed_precision': 'bf16',
    '--num_cpu_threads_per_process': 1,
}

task_dic = {
    "wan_t2v_13":" t2v-1.3B",
    "wan_t2v": "t2v-14B",
    "wan_i2v": "i2v-14B"}

def data_config(resolution, extension, batch_size):
  return {
    "general": {
        "resolution": resolution,
        "caption_extension": extension,
        "batch_size": batch_size,
        "enable_bucket": True,
        "bucket_no_upscale": False
    },
    "datasets": []
  }

def config(model_train, optimizer_type, learning_rate, lr_scheduler, network_dim, network_alpha, max_train_epochs, save_every_n_epochs, save_last_n_epochs, save_every_n_steps, output_dir, output_name):
  dataset_config_path = os.path.join(root_dir,"dataset.toml")
  return {
    "hv_train_network.py": True if "hun" in model_train else False,
    "wan_train_network.py": True if "wan" in model_train else False,
    "--task": task_dic[model_train] if "wan" in model_train else False,
    "--dit": f'"{model_path(model_train)}"',
    "--dataset_config": f'"{dataset_config_path}"',
    "--sdpa": True,
    "--flash_attn": False,
    "--flash3": False,
    "--sage_attn": False,
    "--xformers": False,
    "--split_attn": True,
    "--mixed_precision": "bf16",
    "--fp8_base": True,
    "--optimizer_type": optimizer_type,
    "--optimizer_args": False,
    "--learning_rate": learning_rate,
    "--lr_scheduler": lr_scheduler,
    "--gradient_checkpointing": True,
    "--gradient_accumulation_steps": False,
    "--max_data_loader_n_workers": 2,
    "--persistent_data_loader_workers": True,
    "--network_module": "networks.lora" if "hun" in model_train else "networks.lora_wan",
    "--network_dim": network_dim,
    "--network_alpha": network_alpha,
    "--network_dropout": False,
    "--network_weights": False,
    "--timestep_sampling": "shift",
    "--discrete_flow_shift": 7.0 if "hun" in model_train else 3.0,
    "--max_train_steps": False,
    "--max_train_epochs": max_train_epochs,
    "--save_every_n_epochs": save_every_n_epochs,
    "--save_last_n_epochs": False if save_last_n_epochs <= 0 else save_last_n_epochs,
    "--save_every_n_steps": False if save_every_n_steps <= 0 else save_every_n_steps,
    "--seed": 42,
    "--output_dir": f'"{output_dir}/{output_name}_{model_train}"',
    "--output_name": f'"{output_name}"'
}

sample_prompt_path = os.path.join(root_dir,"prompt.txt")

def extra(model_train, sample_every_n_steps, wandb_api_key, output_name, save_state, save_state_on_train_end, resume, author, description):
  return {
    "--vae": f'"{model_path("vae_wan")}"' if "wan" in model_train else f'"{model_path("vae_hun")}"',
    "--t5": f'"{model_path("tt5_clip")}"' if "wan" in model_train else False,
    "--text_encoder1": f'"{model_path("hun_text_encoder1")}"' if "hun" in model_train else False,
    "--text_encoder2": f'"{model_path("hun_text_encoder2")}"' if "hun" in model_train else False,
    "--clip": f'"{model_path("wan_clip_vision")}"' if "wan" in model_train else f'"{model_path("hun_clip_vision")}"',
    "--sample_prompts": f'"{sample_prompt_path}"',
    "--sample_every_n_epochs": False,
    "--sample_every_n_steps": sample_every_n_steps,
    "--sample_at_first": True if sample_every_n_steps > 0 else False,
    "--log_with": "wandb" if wandb_api_key != "" else False,
    "--wandb_api_key": wandb_api_key if wandb_api_key != "" else False,
    "--wandb_run_name": f'"{model_train}_{output_name}"',
    "--save_state": save_state,
    "--save_state_on_train_end": save_state_on_train_end,
    "--resume": resume if resume != "" else False,
    '--metadata_title': f'"{output_name}"',
    '--metadata_author': f'"{author}"',
    '--metadata_description': f'"{description}"',
    '--metadata_license': False, # "MIT"
    '--metadata_tags': False, # "sdvn.me"
}

def prompt(model_train, sample_prompt, folder_train, sample_image_i2v_path):
  sample_image_i2v_path = os.path.join(repo_dir, "example.png") if sample_image_i2v_path == "" else sample_image_i2v_path
  if "i2v" in model_train:
    prompt = f"""{sample_prompt if sample_prompt != "" else random_sample(folder_train)} --w {sample_size.split(",")[0]} --h {sample_size.split(",")[1]} --f 25 --i {sample_image_i2v_path}"""
  else:
    prompt = f"""{sample_prompt if sample_prompt != "" else random_sample(folder_train)} --w {sample_size.split(",")[0]} --h {sample_size.split(",")[1]} --f 25"""
  write_file(sample_prompt_path,prompt)

def dataset_file(Image_train, Video_train, image_dir, data_config, video_dir, model_train, num_repeats, frame_extraction, target_frames, frame_stride, frame_sample, max_frames):
  dataset = os.path.join(root_dir,"dataset.toml")
  if Image_train:
    for dir in check_sub_dir(image_dir):
      data_config["datasets"].append({
            "image_directory": dir,
            "cache_directory": os.path.join(root_dir,f"cache_{model_train}"),
            "num_repeats": repeat_dir(dir,num_repeats),
            })
  if Video_train:
    for dir in check_sub_dir(video_dir):
      data_config["datasets"].append({
            "video_directory": dir,
            "cache_directory": os.path.join(dir,f"cache_{model_train}"),
            "num_repeats": repeat_dir(dir,num_repeats),
            "frame_extraction": frame_extraction,
            "target_frames": target_frames,
            "frame_stride": frame_stride,
            "frame_sample": frame_sample,
            "max_frames": max_frames,
        })

  with open(dataset, "w") as file:
      toml.dump(data_config, file)

def cache_latents_config(model_train):
  clip_vision_path = model_path("wan_clip_vision") if "wan" in model_train else model_path("hun_clip_vision")
  return {
    "python": True,
    "cache_latents.py": True if "hun" in model_train else False,
    "wan_cache_latents.py": True if "wan" in model_train else False,
    "--dataset_config": f'"{dataset_config_path}"',
    "--vae": f'"{model_path("vae_wan")}"' if "wan" in model_train else f'"{model_path("vae_hun")}"',
    "--vae_chunk_size": 32 if "hun" in model_train else False,
    "--vae_tiling": True if "hun" in model_train else False,
    "--clip": f'"{clip_vision_path}"' if "i2v" in model_train else False,
}

def cache_text_encoder_config(model_train):
  return {
    "python": True,
    "cache_text_encoder_outputs.py": True if "hun" in model_train else False,
    "wan_cache_text_encoder_outputs.py": True if "wan" in model_train else False,
    "--dataset_config": f'"{dataset_config_path}"',
    "--text_encoder1": f'"{model_path("hun_text_encoder1")}"' if "hun" in model_train else False,
    "--text_encoder2": f'"{model_path("hun_text_encoder2")}"' if "hun" in model_train else False,
    "--t5": f'"{model_path("tt5_clip")}"' if "wan" in model_train else False,
    "--batch_size": 16
}